In [ ]:
#==================================================================
#Program: blockSegmentation
#Version: 1.0
#Author: David Helminiak
#Date Created: September 20, 2024
#Date Last Modified: September 21, 2024
#Description: Load WSI and visualize different block segmentation methods/parameters
#Operation: Move back into main program directory before running.
#==================================================================

#Have the notebook fill more of the display width
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:80% !important; }</style>"))

#RNG seed value to ensure run-to-run consistency (-1 to disable)
manualSeedValue = 0

#Should warnings and info messages be shown during operation
debugMode = True

#Should progress bars be visualized with ascii characters
asciiFlag = False

#When splitting WSI images, what size should the resulting blocks be (default: 400)
#Should remain consistent with block sizes given for training
blockSize = 400

#What weight should be used when overlaying data
overlayWeight = 0.5

#What is the camera resolution in mm/pixel for the instrument that acquired the data being used
cameraResolution = 0.00454

#Load external libraries
exec(open("./CODE/EXTERNAL.py", encoding='utf-8').read())

#Colors for background and blocks identified as foreground data
cmapBlock = colors.ListedColormap(['black', 'red'])

#Load/synchronize data labeling, drop excluded rows, and extract relevant metadata
def loadMetadata_WSI(filename):
    metadata = pd.read_csv(filename, header=0, names=['Sample', 'Label'], converters={'Sample':str, 'Label':str})
    return metadata['Sample']

def processWSI(filename):
    
    #Load WSI
    imageWSI = cv2.cvtColor(cv2.imread(filename, cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB)
    
    #Crop WSI to the foreground area using Otsu
    imageWSI_gray = cv2.cvtColor(imageWSI, cv2.COLOR_RGB2GRAY)
    x, y, w, h = cv2.boundingRect(cv2.threshold(imageWSI_gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]) 
    imageWSI = imageWSI[y:y+h, x:x+w]
    imageWSI_gray = imageWSI_gray[y:y+h, x:x+w]

    #Pad the image as needed (as symmetrially as possible) for an even division by the specified block size; compute numBlocks per row/column
    padHeight = (int(np.ceil(imageWSI.shape[0]/blockSize))*blockSize)-imageWSI.shape[0]
    padWidth = (int(np.ceil(imageWSI.shape[1]/blockSize))*blockSize)-imageWSI.shape[1]
    padTop, padLeft = padHeight//2, padWidth//2
    padBottom, padRight = padTop+(padHeight%2), padLeft+(padWidth%2)
    imageWSI = np.pad(imageWSI, ((padTop, padBottom), (padLeft, padRight), (0, 0)))
    imageWSI_gray = np.pad(imageWSI_gray, ((padTop, padBottom), (padLeft, padRight)))
    numBlocksRow, numBlocksCol = imageWSI.shape[0]//blockSize, imageWSI.shape[1]//blockSize
    overlayBase = copy.deepcopy(imageWSI)

    #Split the WSI (color and grayscale) into blocks and flatten
    imageWSI = imageWSI.reshape(numBlocksRow, blockSize, numBlocksCol, blockSize, imageWSI.shape[2]).swapaxes(1,2)
    imageWSI = imageWSI.reshape(-1, imageWSI.shape[2], imageWSI.shape[3], imageWSI.shape[4])
    imageWSI_gray = imageWSI_gray.reshape(numBlocksRow, blockSize, numBlocksCol, blockSize).swapaxes(1,2)
    imageWSI_gray = imageWSI_gray.reshape(-1, imageWSI_gray.shape[2], imageWSI_gray.shape[3])

    return imageWSI, imageWSI_gray, overlayBase, numBlocksRow, numBlocksCol
    
#Directory/file specification
dir_data = '.' + os.path.sep + 'DATA' + os.path.sep
dir_blocks_data = dir_data + 'BLOCKS' + os.path.sep
dir_blocks_inputWSI = dir_blocks_data + 'INPUT_WSI' + os.path.sep
file_blocks_metadataWSI = dir_blocks_inputWSI + 'metadata_WSI.csv'
dir_WSI_data = dir_data + 'WSI' + os.path.sep
dir_WSI_inputs = dir_WSI_data + 'INPUT_WSI' + os.path.sep
file_WSI_metadataWSI = dir_WSI_inputs + 'metadata_WSI.csv'

#Load metadata for WSI
sampleNames_blocks = loadMetadata_WSI(file_blocks_metadataWSI)
WSIFilenames_blocks = np.asarray([dir_blocks_inputWSI + sampleName + '.jpg' for sampleName in sampleNames_blocks])
sampleNames_WSI = loadMetadata_WSI(file_WSI_metadataWSI)
WSIFilenames_WSI = np.asarray([dir_WSI_inputs + sampleName + '.jpg' for sampleName in sampleNames_WSI])
#sampleNames, WSIFilenames = sampleNames_blocks, WSIFilenames_blocks
#sampleNames, WSIFilenames = sampleNames_WSI, WSIFilenames_WSI
sampleNames, WSIFilenames = np.concatenate([sampleNames_blocks, sampleNames_WSI]), np.concatenate([WSIFilenames_blocks, WSIFilenames_WSI])

#Compute otsu threshold for the WSI
computeOtsu = False
if computeOtsu:
    otsuThresholds = []
    for filename in tqdm(WSIFilenames, desc='Otsu', leave=True, ascii=asciiFlag):
        imageWSI_gray = cv2.cvtColor(cv2.imread(filename, cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2GRAY)
        otsuThreshold = cv2.threshold(imageWSI_gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[0]
        otsuThresholds.append(otsuThreshold)
    otsuThresholds = np.asarray(otsuThresholds)

#Minimum value [0, 255] for a grayscale pixel to be considered as a foreground location during block extraction
#Determined as minimum Otsu threshold across all available WSI
blockBackgroundValue = otsuThresholds.min()
print('Determined blockBackgroundValue:', blockBackgroundValue)

#print('Otsu - Min: '+str(otsuThresholds.min())+' Max: '+str(otsuThresholds.max())+' Mean: '+str(otsuThresholds.mean())+' Std: '+str(otsuThresholds.std()))

#Otsu threshold data for classifier-training WSI
#Otsu - Min: 21.0 Max: 64.0 Mean: 38.84848484848485 Std: 9.182068203416629

#Otsu threshold data for non-classifier-training WSI
#Otsu - Min: 11.0 Max: 65.0 Mean: 33.95813953488372 Std: 9.993165863755591

#Otsu threshold data for all WSI
#Otsu - Min: 11.0 Max: 65.0 Mean: 35.1067615658363 Std: 10.025376547631357


In [ ]:
#Specify a filename index
index = 3

#What is the minimum area/quantity (mm^2) of foreground data that should qualify a block for classification
#Result should not exceed blockSize*cameraResolution
minimumForegroundArea = 1.0**2

#Load and process indexed WSI
print('Sample: ' + sampleNames[index])
imageWSI, imageWSI_gray, overlayBase, numBlocksRow, numBlocksCol = processWSI(WSIFilenames[index])

#Compute the ratio of pixel count to total area that should be required for a block to be considered as potentially holding foreground data (1mm^2 occupied area)
#blockBackgroundRatio = ((minimumForegroundArea/cameraResolution)**2)/(blockSize**2)
blockBackgroundRatio = minimumForegroundArea/((blockSize*cameraResolution)**2)
if blockBackgroundRatio > 1.0: sys.exit('\nError - Minimum foreground area specified for foreground data exceeds the given block size.')

#Determine blocks that would be considered to hold foreground data
blockMap = np.zeros(overlayBase.shape[:2])
blockIndex = 0
for rowNum in range(0, numBlocksRow):
    for colNum in range(0, numBlocksCol):
        if np.mean(imageWSI_gray[blockIndex] >= blockBackgroundValue) >= blockBackgroundRatio: 
            locationRow, locationColumn= rowNum*blockSize, colNum*blockSize
            blockMap[locationRow:locationRow+blockSize, locationColumn:locationColumn+blockSize] = 1
        blockIndex += 1

#Show locations of blocks that would be extracted/classified
blockMap = cmapBlock(blockMap)[:,:,:3].astype(np.uint8)*255
overlayMap = cv2.addWeighted(overlayBase, 1.0, blockMap, overlayWeight, 0.0)
plt.figure(figsize=(10,10))
plt.imshow(overlayMap)
plt.show()
plt.close()



In [ ]:
#For debugging/development
#Select, visualize, and evaluate a specific block from the WSI

#Specify a row and column
rowNum, colNum = 0, 12

#Extract and visualize block portion that exceeds the background threshold
locationRow, locationColumn= rowNum*blockSize, colNum*blockSize
overlayBase_gray = cv2.cvtColor(overlayBase, cv2.COLOR_RGB2GRAY)
blockImage = overlayBase_gray[locationRow:locationRow+blockSize, locationColumn:locationColumn+blockSize]
plt.figure(figsize=(5,5))
plt.imshow(blockImage >= blockBackgroundValue, cmap='gray')
plt.show()
plt.close()
valueMean = np.mean(blockImage >= blockBackgroundValue)
print('Foreground/Background Ratio', np.mean(valueMean))
if valueMean >= blockBackgroundRatio: print('Block would be extracted')
else: print('Block would not be extracted')